In [ ]:
%%capture
pip install cobra

In [ ]:
modelpath=  "/content/drive/MyDrive/Halomonas_Model_devt/HalomonasmodelV1.xml"

In [ ]:
model = cobra.io.read_sbml_model = (modelpath)

In [ ]:
model

In [ ]:
model.summary()

Now that we have a grasp of the model, lets look at the gene knockout predictions

In [ ]:
#import the necessary packages
import pandas
from time import time

import cobra.test
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

In [ ]:
#change the objective from biomass to PHB production

#1. check the objective first
biomass_rxn = model.reactions.get_by_id("Biomass_Ecoli_core") #the objective (the biomass reaction), is that with an linear coefficient of 1.

from cobra.util.solver import linear_reaction_coefficients
linear_reaction_coefficients(model)

#The objective function can be changed by assigning Model.objective, 
#which can be a reaction object (or just it’s name), or a dict of {Reaction: objective_coefficient}.

# change the objective to ATPM
model.objective = "ATPM"

# The upper bound should be 1000, so that we get
# the actual optimal value
model.reactions.get_by_id("ATPM").upper_bound = 1000.
linear_reaction_coefficients(model)


In [ ]:
#Cheeck the new optimal value
model.optimize().objective_value

In [ ]:
#FVA
#FBA will not give always give unique solution, because multiple flux states can achieve the same optimum. 
#FVA (or flux variability analysis) finds the ranges of each metabolic flux at the optimum

from cobra.flux_analysis import flux_variability_analysis

flux_variability_analysis(model, model.reactions[:10])


from cobra.flux_analysis import flux_variability_analysis
flux_variability_analysis(model, model.reactions[:10])


#loopless FBA to account for mathematicall loops that do not occur invivo
flux_variability_analysis(model, reaction_list=loop_reactions, loopless=True)

#Using FVA summary to check for optimum or specfic reactions
model.metabolites.pyr_c.summary(fva=0.95)



In [ ]:
#Knocking out single reactions
print('complete model: ', cobra_model.optimize())
with cobra_model:
    cobra_model.reactions.PFK.knock_out()
    print('pfk knocked out: ', cobra_model.optimize())

In [ ]:
#Knocking out single genes
print('complete model: ', cobra_model.optimize())
with cobra_model:
    cobra_model.genes.b1723.knock_out()
    print('pfkA knocked out: ', cobra_model.optimize())
    cobra_model.genes.b3916.knock_out()
    print('pfkB knocked out: ', cobra_model.optimize())

In [ ]:
#Perform all single gene deletions on a model
deletion_results = single_gene_deletion(cobra_model)

In [ ]:
#Perform all single reaction deletions on a model
single_reaction_deletion(cobra_model)

In [ ]:
#Double gene deletions
double_gene_deletion(
    cobra_model, cobra_model.genes[-5:]).round(4)

In [ ]:
#Double reaction deletions
double_reaction_deletion(
    cobra_model, cobra_model.reactions[2:7]).round(4)

In [ ]:
#Accessing individual deletion results

#reactions 
single = single_reaction_deletion(cobra_model)
double = double_reaction_deletion(cobra_model)

print(single.knockout["ATPM"])
print(double.knockout[{"ATPM", "TKT1"}])


#Genes
atpm = cobra_model.reactions.ATPM
tkt1 = cobra_model.reactions.TKT1
pfk = cobra_model.reactions.PFK

print(single.knockout[atpm, tkt1, pfk])
print(double.knockout[{atpm, tkt1}, {atpm, pfk}, {atpm}])